In [2]:
from bqplot import pyplot as plt
import ipywidgets as widgets
import numpy as np
import matplotlib.pyplot as mplt
%matplotlib inline

In [3]:
# generate some fake 
n = 2000
x = np.linspace(0.0, 10.0, n)
np.random.seed(0)
y = np.cumsum(np.random.randn(n)*10).astype(int)

In [4]:
fig_hist = plt.figure( title='Histogram')
hist = plt.hist(y, bins=25)

In [5]:
hist.bins = 10;

In [6]:
slider = widgets.IntSlider(description='Bins number', min=1, max=100, v_model=30)

In [7]:
widgets.link((hist, 'bins'), (slider, 'value'))

fig_lines = plt.figure( title='Line Chart')
lines = plt.plot(x, y)

fig_lines.layout.width = 'auto'
fig_lines.layout.height = 'auto'
fig_hist.layout.width = 'auto'
fig_hist.layout.height = 'auto'

grid_layout = widgets.GridspecLayout(5, 3)

grid_layout[:2, :] = fig_lines
grid_layout[2:4, :] = fig_hist
grid_layout[4, 1] = slider

grid_layout.layout.height = '1000px'

grid_layout

GridspecLayout(children=(Figure(axes=[Axis(scale=LinearScale()), Axis(orientation='vertical', scale=LinearScal…

In [8]:
selector = plt.brush_int_selector()
def update_range(*ignore):
    if selector.selected is not None and len(selector.selected) == 2:
        xmin, xmax = selector.selected
        mask = (x > xmin) & (x < xmax)
        hist.sample = y[mask]
selector.observe(update_range, 'selected')        

In [9]:
def plot_func(a, f):
    mplt.figure(2)
    x = np.linspace(0, 2*np.pi, num=1000)
    y = a*np.sin(1/f*x)
    mplt.plot(x,y)
    mplt.ylim(-1.1, 1.1)
    mplt.title('a sin(f)')
    mplt.show()

interactive_plot = widgets.interactive(plot_func, a=(-1,0,0.1), f=(0.1, 1))
output = interactive_plot.children[-1]
output.layout.height = '300px'
interactive_plot

interactive(children=(FloatSlider(value=-1.0, description='a', max=0.0, min=-1.0), FloatSlider(value=0.55, des…